In [1]:
import os
from pathlib import Path
import sys

import pandas as pd


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.utils.data_extracting import (
    check_income_or_expense,
    extract_text_from_pdf,
    get_acc_files_from_gdrive_folder,
    get_all_transactions,
    get_balance_of_account,
    get_transaction_value,
)
from src.utils.google_services import set_up_google_connection

# Functions

# Setup

In [2]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')
client, service = set_up_google_connection(credentials_path)

Using local JSON credentials


In [3]:
SPREADSHEET_ID = os.environ.get('SPREADSHEET_ID')
TEMP_FOLDER_ID = os.environ.get('TEMP_FOLDER_ID')
REGULAR_FOLDER_ID = os.environ.get('REGULAR_FOLDER_ID')

In [4]:
acc_states = get_acc_files_from_gdrive_folder(TEMP_FOLDER_ID, service)
acc_states

[{'id': '1EdnDpB38JUnIFEsowTZdnrduUadhZLmk',
  'name': '2115089016_2023_Nr.002_Kontoauszug.pdf'}]

In [5]:
file_id = acc_states[0]['id']
text = extract_text_from_pdf(file_id, service=service)
lines = text.split('\n')
len(lines)

126

In [6]:
acc_balance_old = get_balance_of_account(lines, 'alter Kontostand')
acc_balance_new = get_balance_of_account(lines, 'neuer Kontostand')
print(f'Old acc value, Value: {acc_balance_old[0]}€ - Line-Index: {acc_balance_old[1]}')
print(f'New acc value, Value: {acc_balance_new[0]}€ - Line-Index: {acc_balance_new[1]}')

all_transactions = get_all_transactions(lines, acc_balance_old[1], acc_balance_new[1])
print(f'Count of transactions: {len(all_transactions)}')

Old acc value, Value: 2972.89€ - Line-Index: 11
New acc value, Value: 3427.84€ - Line-Index: 94
Count of transactions: 16


# Open Spreadsheet

In [7]:
year = acc_states[0]['name'].split('_')[1]
print(year)
SPREADSHEET_ID = os.environ.get('SPREADSHEET_ID')
print(f'Using spreadsheet ID: {SPREADSHEET_ID}')

2023
Using spreadsheet ID: 1z_rr8nVFlF3PKIbO4v6xv_tSswop7sJRzRftzWzPdE8


In [8]:
spreadsheet = client.open_by_key(SPREADSHEET_ID)

sheet_transactions = spreadsheet.worksheet('Transaktionen')
df_sheet_transactions = pd.DataFrame(sheet_transactions.get_all_values())

In [9]:
df_sheet_transactions.columns = df_sheet_transactions.iloc[0]
df_sheet_transactions = df_sheet_transactions[1:].reset_index(drop=True)

In [10]:
df_sheet_transactions.head()

,Wer,Betrag,Art,Datum,Kategorie,Hinweis
0,Elisabeth Heuser,"507,60",Ausgabe,2023-01-02,,
1,Freundeskreis Elisabeth-Hospiz e.V.,"10,00",Ausgabe,2023-01-02,,
2,KAUFLAND,"10,37",Ausgabe,2023-01-02,,
3,11teamsports Koeln GmbH,"254,00",Ausgabe,2023-01-02,,
4,Heinz-Dieter Heuser,"60,00",Einnahme,2023-01-03,,


# Add transactions to GSheet -> Change!

In [15]:
for transaction_index in range(len(all_transactions)):
    transaction = all_transactions[transaction_index]

    print(transaction)

    transaction_value = get_transaction_value(transaction)
    print(transaction_value)
    transaction_type = check_income_or_expense(transaction)
    print(transaction_type)
    name = transaction[1].strip()
    print(name)
    date_str = str(transaction[0].split(' ')[0] + year)
    date = pd.to_datetime(date_str, format='%d.%m.%Y').date()
    print(date)

    df_sheet_transactions.loc[len(df_sheet_transactions)] = [
        name,
        transaction_value,
        transaction_type,
        date,
        None,
        None,
    ]  # Neue Zeile

['01.02. 01.02. Dauerauftragsbelast 10,00 S', 'Freundeskreis Elisabeth-Hospiz e.V.', 'Freundeskreis Elisabeth-Hospiz e.V.Spende /*DA-2* IBAN: DE70370695202107385013 BIC: GENODED1RST']
10.0
Ausgabe
Freundeskreis Elisabeth-Hospiz e.V.
2023-02-01
['02.02. 02.02. Kartenzahlung girocard 61,14 S', 'Mundorf', 'MundorfMundorf Tank 18/Siegburg/DE01.02.2023 um 18:55:35 Uhr 74054122/152785/ECTL/NPIN37069520/2115089016/1/1225 REF 247632/260047']
61.14
Ausgabe
Mundorf
2023-02-02
['03.02. 03.02. Dauerauftragsgutschr 60,00 H', 'Heinz-Dieter Heuser', 'Heinz-Dieter HeuserTaschengeld']
60.0
Einnahme
Heinz-Dieter Heuser
2023-02-03
['06.02. 06.02. Überweisungsgutschr. 9,00 H', 'Florian Peter Bratka', 'Florian Peter BratkaJulius Geburtstagsgeschenk']
9.0
Einnahme
Florian Peter Bratka
2023-02-06
['10.02. 10.02. Dauerauftragsbelast 10,00 S', 'Elisabeth Heuser', 'Elisabeth HeuserDAZN /*DA-1* IBAN: DE41500105175409388951 BIC: INGDDEFFXXX']
10.0
Ausgabe
Elisabeth Heuser
2023-02-10
['13.02. 13.02. Kartenzahlung 

In [12]:
df_sheet_transactions

,Wer,Betrag,Art,Datum,Kategorie,Hinweis
0,Elisabeth Heuser,"507,60",Ausgabe,2023-01-02,,
1,Freundeskreis Elisabeth-Hospiz e.V.,"10,00",Ausgabe,2023-01-02,,
2,KAUFLAND,"10,37",Ausgabe,2023-01-02,,
3,11teamsports Koeln GmbH,"254,00",Ausgabe,2023-01-02,,
4,Heinz-Dieter Heuser,"60,00",Einnahme,2023-01-03,,
5,ARAL AG,"61,03",Ausgabe,2023-01-09,,
6,Elisabeth Heuser,"10,00",Ausgabe,2023-01-10,,
7,PayPal Europe S.a.r.l. et Cie S.C.A,"15,99",Ausgabe,2023-01-11,,
8,Xaver Heuser,"200,00",Ausgabe,2023-01-13,,
9,HENNES OPTIK + HRGE LOHMAR,"58,00",Ausgabe,2023-01-13,,


In [13]:
from src.utils.data_loading import update_google_sheet


update_google_sheet(sheet_transactions, df_sheet_transactions)

In [14]:
from src.utils.google_services import move_file


move_file(file_id, acc_states[0]['name'], TEMP_FOLDER_ID, REGULAR_FOLDER_ID, service)

File 1EdnDpB38JUnIFEsowTZdnrduUadhZLmk moved to 1e0FsZObsGaB4aoee2UTftyjWW75AfOrT
